In [1]:
import os
import json
import numpy as np
from pandas import DataFrame
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pathlib
import gzip
import itertools
import random
from collections import namedtuple
from collections import Counter
from scipy import stats
from scipy.stats import norm

%matplotlib inline
plt.rcParams['figure.figsize'] = [16.5, 5]
plt.rcParams['font.size'] = 12
plt.rcParams['pdf.fonttype'] = 42

In [2]:
import vcf

### Define paths to input files and analysis parameters

In [4]:
vcf_path = "/pbi/flash/tmokveld/tmp/egor_ashg_analysis/update/merge.vcf.gz" # A VCF generated by `trgt merge`
min_allele_mult = 5  # Minimum allele multiplicity (remove rare alleles)
min_alleles_per_tr = 500  # TRs with too few alleles will be removed
min_cpgs_to_call_meth = 5 # Ignore allele methylation if it has fewer CpGs than this
min_alleles_with_meth = 250 # Ignore TR methylation if methylation is called in fewer alleles than this

Prepare output paths

In [5]:
pathlib.Path("output/").mkdir(exist_ok=True)
pathlib.Path("scratch/").mkdir(exist_ok=True)

### Summarize merged VCF into a table

In [6]:
def get_allele_recs(tr_rec):
    all_alleles = []
    for alleles in tr_rec.gts.values():
        all_alleles.extend(alleles)
    return all_alleles


def num_samples_with_gt(tr_rec):
    return len([als for als in rec.gts.values() if als])

In [7]:
def get_num_distinct_alleles(alleles):
    score = len(set(a.seq for a in alleles))
    return score

In [8]:
def get_frequent_alleles(alleles, min_allele_mult):
    seqs = [a.seq for a in alleles]
    freq_seqs = set(seq for seq, count in Counter(seqs).items() if count >= min_allele_mult)
    return [a for a in alleles if a.seq in freq_seqs]


def get_top_seq(alleles):
    seqs = [a.seq for a in alleles]
    counted_seqs = list(Counter(seqs).items())
    counted_seqs.sort(key=lambda rec: rec[1], reverse=True)
    return counted_seqs[0][0]


def get_median_meth(alleles):
    meths = [a.meth for a in alleles if a.meth is not None and a.seq.count("CG") >= min_cpgs_to_call_meth]
    if len(meths) >= min_alleles_with_meth:
        return np.median(meths), np.std(meths)
    else:
        return None, None

    
def get_median_len(alleles):
    lens = [len(a.seq) for a in alleles]
    return np.median(lens), np.std(lens)


def get_tr_type(rec):
    if rec.trid.count(",") > 0:
        return "vc"
    
    fields = rec.trid.split("-")
    if len(fields) != 4:
        return None
    chrom, start, end, motif = fields
    start, end = int(start), int(end)

    left_extension = abs(rec.locus.start - start)
    right_extension = abs(rec.locus.end - end)
    if left_extension > 5 or right_extension > 5:
        return "vc"
    
    return "solo"

In [9]:
def encode(value):
    if value is None:
        return "NA"
    if type(value) == int:
        return str(value)
    if type(value) in [float, np.float64]:
        return f"{value:.2f}"
    assert False, f"Unexpected value '{value}' ({type(value)})"

In [ ]:
num_skipped = 0

with open("output/summary_table.tsv", "w") as outfile:
    for index, rec in enumerate(vcf.get_rec(vcf_path)):
        alleles = get_allele_recs(rec)
        alleles = get_frequent_alleles(alleles, min_allele_mult)
        if len(alleles) < min_alleles_per_tr:
            num_skipped += 1
            continue

        tr_type = get_tr_type(rec)
        if tr_type is None:
            print(f"Skipping {rec.trid}")
            continue

        top_seq = get_top_seq(alleles)
        len_median, len_std = get_median_len(alleles)
        num_alleles = len(alleles)
        num_distinct_alleles = get_num_distinct_alleles(alleles)
        meth_median, meth_std = get_median_meth(alleles)
    
        fields = [rec.trid, tr_type, top_seq, str(num_alleles), str(num_distinct_alleles),
                 encode(len_median), encode(len_std), encode(meth_median), encode(meth_std)]
        print("\t".join(fields), file=outfile)



print(f"TRs processed: {index + 1}")
print(f"Number of TRs skipped: {num_skipped} ({100 * num_skipped / (index + 1):.2f}%)")

Skipping VWA1
Skipping DAB1
Skipping ABCD3
Skipping NOTCH2NLA
Skipping NOTCH2NLC
Skipping NUTM2B-AS1
Skipping C11ORF80
Skipping CBL
Skipping ATN1
Skipping DIP2B
Skipping ATXN2
Skipping RILPL1
Skipping ZIC2
Skipping PABPN1
